In [1]:
import tkinter
from tkinter import *
from tkinter.ttk import Combobox
from tkinter import scrolledtext
from tkinter.ttk import Progressbar
from tkinter import ttk
from tkinter import messagebox
import pandas as pd
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from keras.models import load_model
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.optimizers import Adam
from keras.callbacks import LambdaCallback
from tqdm import tqdm
import time
from keras.models import model_from_json
import string

In [2]:
# Model is trained to take 30 chars as input
INPUT_TEXT_LEN_FIXED = 20 

# This is the size of characters of output lyrics
OUTPUT_CHARS = 400

# Lyrics Alphabet Dictionary
chars = [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

# Create a dictionary mapping from character to index 
char_to_index = dict((c, i) for i, c in enumerate(chars))

# Create a dictionary mapping from index to character 
index_to_char = dict((i, c) for i, c in enumerate(chars))

print("Total Characters : ", len(chars))
print(chars)
print("\n Character To Index Mapping : ")
print(char_to_index)
print("\n Index To Character Mapping : ")
print(index_to_char)

Total Characters :  27
[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

 Character To Index Mapping : 
{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}

 Index To Character Mapping : 
{0: ' ', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [3]:
class LyricsGenerator:
    def __init__(self, win):
                
        # welcome message
        self.lbl=Label(win, text="Welcome to Lyrics Generator", fg='black', font=("Helvetica", 16))
        self.lbl.place(x=210, y=50)
        
        # pick a genre message
        self.genre_lbl=Label(win, text="Select a music genre:", fg='black', font=("Helvetica", 12))
        self.genre_lbl.place(x=150, y=100)
        
        # dropdown list for genres selection
        self.combo = Combobox(win)
        self.combo['values']= ('pop', 'blues', 'hiphop', 'country', 'rock')
        self.combo.current(0) #set the selected item
        self.combo.place(x=330, y=100)
        
        # prompt input message
        self.seed_lbl = Label(win, text="Enter a few words to get started: ", fg='black', font=("Helvetica", 12))
        self.seed_lbl.place(x=150, y=160)
        
        # seed 
        self.user_input=Entry(win, bd=5,width=70)
        self.user_input.place(x=150, y=200)
        
        # progress bar
        self.style = ttk.Style()
        self.style.theme_use('default')
        self.style.configure("black.Horizontal.TProgressbar", background='black')
        self.bar = Progressbar(win, length=450, style='black.Horizontal.TProgressbar')
        self.bar.place(x=150,y=280)

        # button
        self.btn = Button(win, text="Generate" ,command=self.generate_lyrics)
        self.btn.place(x=330,y=230)
        
        self.reset = Button(win,text='Reset',command=self.clear)
        self.reset.place(x=400,y=230)
        
        # output box
        self.output = scrolledtext.ScrolledText(win,width=55,height=10)
        self.output.grid(column=0,row=0)
        self.output.place(x=150,y=300)
        
    
    # helper function to sample an index from a probability array
    def sample(self,preds, temperature=1.0):
        epsilon = 1e-08
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds + epsilon) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)
    
    def predict_lyrics(self,seed):

        diversity = 0.2
        genre=str(self.combo.get())
        
        if genre=='rock':
            json_file = open('models/rock_model.json', 'r')
            loaded_model_json = json_file.read()
            json_file.close()
            model = model_from_json(loaded_model_json)
            # load weights into new model
            model.load_weights("models/rock_model.h5")
        elif genre=='pop':
            json_file = open('models/pop_model.json', 'r')
            loaded_model_json = json_file.read()
            json_file.close()
            model = model_from_json(loaded_model_json)
            # load weights into new model
            model.load_weights("models/pop_model.h5")
        elif genre=='hiphop':
            json_file = open('models/hiphop_model.json', 'r')
            loaded_model_json = json_file.read()
            json_file.close()
            model = model_from_json(loaded_model_json)
            # load weights into new model
            model.load_weights("models/hiphop_model.h5")
        elif genre=='country':
            json_file = open('models/country_model.json', 'r')
            loaded_model_json = json_file.read()
            json_file.close()
            model = model_from_json(loaded_model_json)
            # load weights into new model
            model.load_weights("models/country_model.h5")
        elif genre=='blues':
            json_file = open('models/blues_model.json', 'r')
            loaded_model_json = json_file.read()
            json_file.close()
            model = model_from_json(loaded_model_json)
            # load weights into new model
            model.load_weights("models/blues_model.h5")
        
        generated = ''
        sentence = seed[:INPUT_TEXT_LEN_FIXED].lower()
        generated += sentence

        for i in tqdm(range(400)):
            x_pred = np.zeros((1, INPUT_TEXT_LEN_FIXED, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_to_index[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = self.sample(preds, diversity)
            next_char = index_to_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
            self.bar['value'] +=0.25
            window.update_idletasks()
            
        return generated
    
    def remove_punc(self,s):  
        exclude = set(string.punctuation)
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        return regex.sub(' ', s)
    
    def generate_lyrics(self):
        seed =str(self.user_input.get())
        seed = self.remove_punc(seed)
        if len(seed)<20:
            messagebox.showerror("Error", "Please enter at least 20 characters, including space and punctuations")
        else:
            generated_lyrics  = self.predict_lyrics(seed)
            self.output.insert(END,generated_lyrics)
    
    def clear(self):
        self.bar['value']=0
        self.user_input.delete(0, "end")
        self.output.delete(1.0,"end")
    
    
        
window=Tk()
mywin=LyricsGenerator(window)
window.title('Lyrics Generator')
window.geometry("700x500")
window.mainloop()

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [00:21<00:00, 18.29it/s]
